<a href="https://colab.research.google.com/github/AshishGusain17/tf-obj-detection/blob/master/ipynb/opencv-videos-tf-v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --user Cython
!pip install --user contextlib2
!pip install --user pillow
!pip install --user lxml
!pip install --user jupyter
!pip install --user matplotlib

In [9]:
!git clone https://github.com/AshishGusain17/tf-obj-detection



Cloning into 'tf-obj-detection'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 34254 (delta 3), reused 19 (delta 1), pack-reused 34233
Receiving objects: 100% (34254/34254), 512.56 MiB | 36.32 MiB/s, done.
Resolving deltas: 100% (22034/22034), done.
Checking out files: 100% (2500/2500), done.


In [10]:
!python /content/models/research/setup.py build
!python /content/models/research/setup.py install

running build
running install
running bdist_egg
running egg_info
creating object_detection.egg-info
writing object_detection.egg-info/PKG-INFO
writing dependency_links to object_detection.egg-info/dependency_links.txt
writing requirements to object_detection.egg-info/requires.txt
writing top-level names to object_detection.egg-info/top_level.txt
writing manifest file 'object_detection.egg-info/SOURCES.txt'
reading manifest file 'object_detection.egg-info/SOURCES.txt'
writing manifest file 'object_detection.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib

creating build
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/EGG-INFO
copying object_detection.egg-info/PKG-INFO -> build/bdist.linux-x86_64/egg/EGG-INFO
copying object_detection.egg-info/SOURCES.txt -> build/bdist.linux-x86_64/egg/EGG-INFO
copying object_detection.egg-info/dependency_links.txt -> build/bdist.linux-x86_64/e

In [11]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -r --recursive '/content/drive/My Drive/colab_obj_detection/faster_rcnn_resnet101_coco_2018_01_28' '/content/models/research/object_detection'
!cp -r --recursive '/content/drive/My Drive/colab_obj_detection/faster_rcnn_inception_resnet_v2_atrous_oid_v4_2018_12_12' '/content/models/research/object_detection'
!cp -r --recursive '/content/drive/My Drive/colab_obj_detection/mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28' '/content/models/research/object_detection'

!cp -r --recursive '/content/drive/My Drive/colab_obj_detection/training' '/content/models/research/object_detection'
!cp -r --recursive '/content/drive/My Drive/colab_obj_detection/images' '/content/models/research/object_detection'

In [12]:

%cd /content/models/research
%set_env PYTHONPATH=/content/models/research:/content/models/research/slim
%cd object_detection/legacy
%cd ..
%cd ..
%cd ..
%cd ..
%cd ..

/content/models/research
env: PYTHONPATH=/content/models/research:/content/models/research/slim
/content/models/research/object_detection/legacy
/content/models/research/object_detection
/content/models/research
/content/models
/content
/


In [7]:
%ls

bin/                                       etc/    opt/    sys/
boot/                                      home/   proc/   tensorflow-1.15.2/
content/                                   lib/    root/   tmp/
datalab/                                   lib32/  run/    tools/
dev/                                       lib64/  sbin/   usr/
dlib-19.18.0-cp27-cp27mu-linux_x86_64.whl  media/  srv/    var/
dlib-19.18.0-cp36-cp36m-linux_x86_64.whl   mnt/    swift/


In [18]:
!pip install -U --pre tensorflow=="2.*"

Requirement already up-to-date: tensorflow==2.* in /usr/local/lib/python3.6/dist-packages (2.2.0rc3)


In [17]:
!pip install pycocotools

In [19]:
!ls
%cd /content/models/research


bin					   etc	  opt	 sys
boot					   home   proc	 tensorflow-1.15.2
content					   lib	  root	 tmp
datalab					   lib32  run	 tools
dev					   lib64  sbin	 usr
dlib-19.18.0-cp27-cp27mu-linux_x86_64.whl  media  srv	 var
dlib-19.18.0-cp36-cp36m-linux_x86_64.whl   mnt	  swift
/content/models/research


In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import os
import pathlib
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [0]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [0]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [0]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

In [0]:
PATH_TO_LABELS = '/content/models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [0]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
# PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/content/models/research/object_detection/test_images')
# TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
# TEST_IMAGE_PATHS

In [100]:
model_name = 'faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28'
detection_model = load_model(model_name)
# http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_2018_01_28.tar.gz
# http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_kitti_2018_01_28.tar.gz
# http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28.tar.gz

672227328/672221478 [==============================] - 9s 0us/step
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [92]:
print(detection_model.inputs)

[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]


In [93]:
detection_model.output_dtypes

{'detection_boxes': tf.float32,
 'detection_classes': tf.float32,
 'detection_scores': tf.float32,
 'num_detections': tf.float32}

In [94]:
detection_model.output_shapes

{'detection_boxes': TensorShape([None, 100, 4]),
 'detection_classes': TensorShape([None, 100]),
 'detection_scores': TensorShape([None, 100]),
 'num_detections': TensorShape([None])}

In [0]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [0]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  # image_np = np.array(Image.open(image_path))
  image_np = np.array(image_path)
  print(image_np.shape)

  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  # display(Image.fromarray(image_np))
  return image_np


In [0]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)

In [105]:
from collections import deque
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import argparse
import cv2
import imutils
import time
from sklearn.metrics import pairwise
from google.colab.patches import cv2_imshow

# cap=cv2.VideoCapture('/content/dog_park.mp4')
cap=cv2.VideoCapture('/content/highway.mp4')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out1 = cv2.VideoWriter('s.avi', fourcc, 30.0, (int(cap.get(3)),int(cap.get(4))))
fps = FPS().start()
time.sleep(2.0)

ctt = 0
while True:
    (grabbed, frame) = cap.read()
    print('frame',frame.shape)

    # frame = imutils.resize(frame, width=600)               this line is actually not working, needs to find reason
    # print('frame_resize',frame.shape)
    print(ctt)
    ctt = ctt + 1
    if ctt > 150:
      break
    frame=show_inference(detection_model, frame)


    # if ctt%20==0:
    #   cv2_imshow( frame)

    out1.write(frame)
    # update the FPS counter
    fps.update()
    key=cv2.waitKey(100)
    if key & 0xFF == ord("q"):
        break

# stop the timer and display FPS information
fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
cap.release()
out1.release()
cv2.destroyAllWindows()

frame (720, 1280, 3)
0
(720, 1280, 3)
frame (720, 1280, 3)
1
(720, 1280, 3)
frame (720, 1280, 3)
2
(720, 1280, 3)
frame (720, 1280, 3)
3
(720, 1280, 3)
frame (720, 1280, 3)
4
(720, 1280, 3)
frame (720, 1280, 3)
5
(720, 1280, 3)
frame (720, 1280, 3)
6
(720, 1280, 3)
frame (720, 1280, 3)
7
(720, 1280, 3)
frame (720, 1280, 3)
8
(720, 1280, 3)
frame (720, 1280, 3)
9
(720, 1280, 3)
frame (720, 1280, 3)
10
(720, 1280, 3)
frame (720, 1280, 3)
11
(720, 1280, 3)
frame (720, 1280, 3)
12
(720, 1280, 3)
frame (720, 1280, 3)
13
(720, 1280, 3)
frame (720, 1280, 3)
14
(720, 1280, 3)
frame (720, 1280, 3)
15
(720, 1280, 3)
frame (720, 1280, 3)
16
(720, 1280, 3)
frame (720, 1280, 3)
17
(720, 1280, 3)
frame (720, 1280, 3)
18
(720, 1280, 3)
frame (720, 1280, 3)
19
(720, 1280, 3)
frame (720, 1280, 3)
20
(720, 1280, 3)
frame (720, 1280, 3)
21
(720, 1280, 3)
frame (720, 1280, 3)
22
(720, 1280, 3)
frame (720, 1280, 3)
23
(720, 1280, 3)
frame (720, 1280, 3)
24
(720, 1280, 3)
frame (720, 1280, 3)
25
(720, 1280,